In [1]:
import numpy as np
import SimpleITK as sitk
import cv2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
np.random.seed(777)
import math
import h5py
import keras
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.applications import VGG19
from keras.applications.inception_v3 import InceptionV3
from keras.applications.nasnet import NASNetMobile
from keras.layers import Input, concatenate, Dense
from keras.layers import GlobalAveragePooling2D

In [2]:
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 1000
num_classes = 1

def load_model():
    input_tensor = Input(shape = input_shape)  

    base_model1=NASNetMobile(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model2=InceptionV3(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model3=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
    base_model4=VGG19(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

    x1 = base_model1.output
    x1 = GlobalAveragePooling2D()(x1)

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    x3 = base_model3.output
    x3 = GlobalAveragePooling2D()(x3)

    x4 = base_model4.output
    x4 = GlobalAveragePooling2D()(x4)

    merge = concatenate([x1, x2, x3 , x4])
    predictions = Dense(1, activation='sigmoid')(merge)

    return Model(inputs=input_tensor,outputs=predictions)

In [3]:
def get_slices_axial_3c(path):
    img = sitk.ReadImage(path, sitk.sitkFloat64)
    arr = sitk.GetArrayFromImage(img)
    arr = arr[49:68,115:200,30:150]

    slices = None
    count = 0

    for i in range(arr.shape[0]):
        slice = arr[arr.shape[0] - i -1, : , : ]
        slice = cv2.resize(slice, (img_height, img_width), interpolation=cv2.INTER_CUBIC)
        slice[slice < 0] = 0
        if slice.max() != 0:
            slice = cv2.merge((slice,slice,slice))
            if count == 0:
                slices = np.array([slice])
            else:
                slices = np.concatenate((slices,[slice]))
        count+=1
    return slices

In [4]:
def evaluate_nii(path):
    model = load_model()
    model.load_weights("./vgg19_axial3c_17012022_1741.h5")
    slices = get_slices_axial_3c(path)
    preds = model.predict(slices)
    return preds.mean()

In [5]:
path = "E:/ESPOL/integradora/desorders/PREPROCESSED/BUY/3018.nii.gz"
slices = get_slices_axial_3c(path)
preds_ICD = model.predict(slices)

In [17]:
path = "E:/ESPOL/integradora/desorders/PREPROCESSED/PD/3012.nii.gz"
slices = get_slices_axial_3c(path)
preds_PD = model.predict(slices)

In [18]:
preds_ICD

array([[9.91874933e-03],
       [2.61160135e-02],
       [6.63598776e-02],
       [1.73572004e-02],
       [1.49181485e-03],
       [1.14426017e-03],
       [1.22937560e-03],
       [4.14103270e-04],
       [1.21377925e-04],
       [3.91624999e-05],
       [5.99771738e-04],
       [2.14362144e-03],
       [1.76984072e-03],
       [9.79512930e-04],
       [2.26408243e-04],
       [1.15093408e-05],
       [1.38149261e-02],
       [2.29406357e-03],
       [2.00808048e-04]], dtype=float32)

In [19]:
preds_PD

array([[0.9999968 ],
       [0.99998355],
       [0.99999535],
       [0.9997914 ],
       [0.9966968 ],
       [0.9334302 ],
       [0.9971144 ],
       [0.99991924],
       [0.9999995 ],
       [0.9974849 ],
       [0.99986255],
       [0.9997214 ],
       [0.9999931 ],
       [0.99957114],
       [0.92820823],
       [0.97823143],
       [0.99650604],
       [0.98137176],
       [0.6471531 ]], dtype=float32)

In [22]:
preds_ICD.mean()

0.007696442

In [23]:
preds_PD.mean()

0.9713174